In [ ]:
from confluent_kafka import Consumer, KafkaError
import json
from s3fs import S3FileSystem

conf = {
    'bootstrap.servers': '3.109.153.20:9092',  
    'group.id': 'my_consumer_group',  
    'auto.offset.reset': 'earliest'   
}
consumer = Consumer(conf)

consumer.subscribe(['demo_stock_project'])

s3 = S3FileSystem()

def consume_messages_and_upload_to_s3(bucket_name):
    print("Waiting for messages...")
    count=0
    while True:  
        msg = consumer.poll(timeout=1.0) 
        if msg is None:
            continue  
        
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Error occurred: {msg.error()}")
                break
        try:
            message_value = json.loads(msg.value().decode('utf-8'))
            key = f"s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_{count}.json" 
            with s3.open(key, 'w') as file:
                json.dump(message_value, file)
            
            print(f"Uploaded message to {key}")
            count+=1
        
        except json.JSONDecodeError as e:
            print(f"Failed to decode JSON: {e} - Received value: {msg.value().decode('utf-8')}")
    consumer.close()
    print("Consumer closed.")
consume_messages_and_upload_to_s3('stock-market-project-test-ukesh-bucket')


Waiting for messages...
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_0.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_1.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_2.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_3.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_4.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_5.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_6.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_7.json
Uploaded message to s3://stock-market-project-test-ukesh-bucket/stock_market_data/stock_market_8.json
Uploaded message to s3://stock-market-project-test-ukesh-b